In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BASE_DIR = '..'
OUTPUTS_DIR = '../outputs'
df_events = pd.read_csv(f'{BASE_DIR}/dataset/events.csv')
df_categories = pd.read_csv(f'{BASE_DIR}/dataset/category_tree.csv')
df_properties = pd.concat([pd.read_csv(f'{BASE_DIR}/dataset/item_properties_part1.csv'), pd.read_csv(f'{BASE_DIR}/dataset/item_properties_part2.csv')])

In [8]:
def missing_values_summary(df, df_name):
    """Calculates and prints the missing values count and percentage for a DataFrame."""
    print(f"# Missing Values in {df_name}")
    missing_count = df.isnull().sum()
    missing_percentage = (missing_count / len(df)) * 100
    missing_df = pd.DataFrame({'Count': missing_count, 'Percentage': missing_percentage})
    print(missing_df[missing_df['Count'] > 0])
    print("\n")

missing_values_summary(df_events, 'df_events')
missing_values_summary(df_properties, 'df_properties')
missing_values_summary(df_categories, 'df_categories')

unique_visitors = df_events['visitorid'].nunique()
unique_items = df_events['itemid'].nunique()
unique_transactions = df_events['transactionid'].nunique()

print("# Unique Counts in df_events")
print(f"Number of unique visitors: {unique_visitors:,}")
print(f"Number of unique items involved in events: {unique_items:,}")
print(f"Number of unique transactions: {unique_transactions:,}")

# Missing Values in df_events
                 Count  Percentage
transactionid  2733644    99.18519


# Missing Values in df_properties
Empty DataFrame
Columns: [Count, Percentage]
Index: []


# Missing Values in df_categories
          Count  Percentage
parentid     25    1.497903


# Unique Counts in df_events
Number of unique visitors: 1,407,580
Number of unique items involved in events: 235,061
Number of unique transactions: 17,672


### Handling missing values

Our primary goal in this preprocessing step is not to simply drop missing data, but to understand why it's missing and handle it intelligently to preserve our valuable behavioral data.

In [9]:
print("Starting missing value handling...")

# 1. Handling df_events
# EDA showed 99.18% of 'transactionid' are NaN.
# This is expected. The 'transactionid' field is ONLY populated for the 'transaction' event type.
# The 99% of missing values represent all the 'view' and 'addtocart' events.
#
# We MUST NOT drop these rows. Dropping them would remove all our behavioral data
# for non-buying sessions, making it impossible to perform our behavioral clustering.
#
# We will fill the NaN values with 0. This makes the column a clean integer type,
# which is easier for processing, and clearly separates non-transaction events (0) from
# actual transaction events (which have a positive ID).
original_event_count = len(df_events)
df_events['transactionid'] = df_events['transactionid'].fillna(0).astype(int)
print(f"Processed 'df_events'. 'transactionid' NaNs filled with 0.")
print(f"Row count remains {len(df_events)} (no rows dropped).")

# 2. Handling df_properties
# EDA showed 0 missing values.
# The data is clean.
# No action required.
print("\nProcessed 'df_properties'. No missing values found.")

# 3. Handling df_categories
# EDA showed 25 missing values for 'parentid'.
# This is also expected. These 25 rows are the "root" categories
# (e.g., "Electronics"). They do not have a parent, so the field is correctly left blank.
#
# We MUST NOT drop these rows, as that would break our category hierarchy.
#
# We will fill these NaNs with 0 to signify
# they are the top-level root nodes. This maintains the integrity of the tree structure.
original_category_count = len(df_categories)
df_categories['parentid'] = df_categories['parentid'].fillna(0).astype(int)
print(f"\nProcessed 'df_categories'. 'parentid' NaNs filled with 0 to represent root categories.")
print(f"Row count remains {len(df_categories)} (no rows dropped).")

print("\nMissing value handling complete.")

Starting missing value handling...
Processed 'df_events'. 'transactionid' NaNs filled with 0.
Row count remains 2756101 (no rows dropped).

Processed 'df_properties'. No missing values found.

Processed 'df_categories'. 'parentid' NaNs filled with 0 to represent root categories.
Row count remains 1669 (no rows dropped).

Missing value handling complete.


### Noise Handling

1. Statistical Noise (Outliers): We saw sessions with 7,000+ events and 50% of visitors with only 1 event. These extremes can pull the cluster centers.

2. Methodological Noise (Irrelevant Data): Sessions without a 'transaction' are essential data for clustering.

**Our Strategy**: We must remove these statistical outliers before we do anything else (like sessionization) because they will severely skew the feature calculations for our K-Means clustering.

In [ ]:
# Step 1: Noise Handling (Statistical Outliers)

print(f"Original df_events shape: {df_events.shape}")

# 1. Identify Outlier Visitors
print("Calculating events per visitor...")
events_per_visitor = df_events.groupby('visitorid')['event'].count()

# The EDA showed a massive skew. We'll define 'noise' as the extreme outliers.
# We can find this threshold programmatically by using a
# high quantile. Let's use the 99.9th percentile as our cutoff.
outlier_threshold = events_per_visitor.quantile(0.999)
print(f"Outlier threshold (99.9th percentile) = {outlier_threshold} events")

# Get the list of visitorids that are above this threshold
outlier_visitors = events_per_visitor[events_per_visitor > outlier_threshold].index
print(f"Found {len(outlier_visitors)} outlier visitors.")


# 2. Filter Outlier Events

# Now, we remove all events associated with these outlier
# visitors from our main df_events dataframe. This ensures they
# won't impact our sessionization or feature engineering.
df_events_cleaned = df_events[~df_events['visitorid'].isin(outlier_visitors)].copy()


# 3. Report Results
original_events = len(df_events)
cleaned_events = len(df_events_cleaned)
events_removed = original_events - cleaned_events
percent_removed = (events_removed / original_events) * 100

print("\n# Noise Handling Summary")
print(f"Original total events: {original_events:,}")
print(f"Cleaned total events: {cleaned_events:,}")
print(f"Removed {events_removed:,} events ({percent_removed:.2f}%) belonging to {len(outlier_visitors)} visitors.")
print(f"Cleaned df_events_cleaned shape: {df_events_cleaned.shape}")

#  This cleaned dataframe is what we'll use for the next step.
# We'll re-assign df_events to this cleaned version for the rest of the pipeline.
df_events = df_events_cleaned

Original df_events shape: (2756101, 5)
Calculating events per visitor...
Outlier threshold (99.9th percentile) = 47.0 events
Found 1352 outlier visitors.

# Noise Handling Summary
Original total events: 2,756,101
Cleaned total events: 2,536,174
Removed 219,927 events (7.98%) belonging to 1352 visitors.
Cleaned df_events_cleaned shape: (2536174, 5)


#### Summary

- We chose Removal/Filtering because the data identified as "noise" was not just "extreme" data; it was anomalous data.
  - Based on the EDAs, the median visitor had only 1 event, while the top 0.1% had over 32, with a max of 7,757.
- Why we removed them instead of using other methods:
  - They Aren't "Extreme Users," They're "Non-Users." A visitor with thousands of events is almost certainly a bot, scraper, or internal test account. This isn't just an "enthusiastic shopper"; it's a different category of user entirely. Our goal is to cluster real customer behavior, so these non-human actors must be removed.
- Why Other Methods Don't Fit:
  - Winsorizing (Capping): This technique is for limiting the influence of a single feature (e.g., capping a session duration at 3 hours). It doesn't make sense for a user. We can't "cap" a bot's behavior to make it look like a human—its entire pattern is wrong.
  - Imputation/Replacement: These methods are for missing data, not bad data. We have no value to "impute" here.
  - Data Smoothing: This is typically for time-series data to reduce signal jitter, which doesn't apply to our task of identifying outlier visitors.
-In short: These visitors are removed because they are not part of the population we want to analyze. Their behavior is statistical "noise" that would contaminate our clusters, pulling the cluster centers away from the real, typical user groups we are trying to find.

## Making visitor sessions

In [11]:
print(f"Starting sessionization on {len(df_events):,} events...")

# 1. Convert Timestamp
# To calculate time differences, we must ensure the 'timestamp'
# column is in datetime format.
if not pd.api.types.is_datetime64_any_dtype(df_events['timestamp']):
    print("Converting timestamp from int (milliseconds) to datetime...")
    df_events['timestamp_dt'] = pd.to_datetime(df_events['timestamp'], unit='ms')
else:
    # Handle case where it might have been converted in a notebook
    df_events['timestamp_dt'] = pd.to_datetime(df_events['timestamp'])

# 2. Sort Values
# Sorting by visitor and time is CRITICAL. This ensures
# that events for the same user are sequential before we calculate
# the time difference between their actions.
print("Sorting events by visitorid and timestamp...")
df_events.sort_values(by=['visitorid', 'timestamp_dt'], inplace=True)

# 3. Calculate Time Difference
# We define a session as a period of activity ending with
# 30 minutes (1800 seconds) of inactivity.
session_timeout_seconds = 30 * 60

# We use groupby('visitorid').diff() to calculate the time (in seconds)
# between consecutive events *for the same visitor*.
# The first event for any visitor will result in 'NaT' (Not a Time).
print("Calculating time difference between events for each visitor...")
df_events['time_diff_seconds'] = df_events.groupby('visitorid')['timestamp_dt'].diff().dt.total_seconds()

# 4. Identify New Sessions
# A new session starts when the time_diff is > our timeout,
# OR when it's the very first event for that user (where time_diff is NaN/NaT).
# We create a boolean flag 'new_session_flag' for this.
df_events['new_session_flag'] = (df_events['time_diff_seconds'] > session_timeout_seconds) | (df_events['time_diff_seconds'].isnull())

# 5. Assign Session IDs
# We use cumsum() (cumulative sum) on the 'new_session_flag'
# boolean (where True=1, False=0). This creates a unique, incrementing ID
# for each new session within that visitor's group.
print("Assigning unique session IDs...")
df_events['session_increment_id'] = df_events.groupby('visitorid')['new_session_flag'].cumsum()

# Create a globally unique session_id by combining the visitorid
# (as a string) with the session_increment_id. This will be our
# primary key for the new session-feature matrix.
df_events['session_id'] = df_events['visitorid'].astype(str) + '_' + df_events['session_increment_id'].astype(str)

# 6. Report Results
session_count = df_events['session_id'].nunique()
visitor_count = df_events['visitorid'].nunique()
avg_sessions_per_visitor = session_count / visitor_count
avg_events_per_session = len(df_events) / session_count

print("\nSessionization Summary")
print(f"Total events processed: {len(df_events):,}")
print(f"Total unique visitors: {visitor_count:,}")
print(f"Total unique sessions created: {session_count:,}")
print(f"Average sessions per visitor: {avg_sessions_per_visitor:.2f}")
print(f"Average events per session: {avg_events_per_session:.2f}")

# Display a sample of the sessionized data to verify
print("\nSample of sessionized data (showing new session flags):")
display(df_events[['timestamp_dt', 'visitorid', 'event', 'time_diff_seconds', 'new_session_flag', 'session_id']].head(10))

# Clean up intermediate columns before next step
df_events.drop(columns=['time_diff_seconds', 'new_session_flag', 'session_increment_id'], inplace=True)

Starting sessionization on 2,756,101 events...
Converting timestamp from int (milliseconds) to datetime...
Sorting events by visitorid and timestamp...
Calculating time difference between events for each visitor...
Assigning unique session IDs...

Sessionization Summary
Total events processed: 2,756,101
Total unique visitors: 1,407,580
Total unique sessions created: 1,761,675
Average sessions per visitor: 1.25
Average events per session: 1.56

Sample of sessionized data (showing new session flags):


,timestamp_dt,visitorid,event,time_diff_seconds,new_session_flag,session_id
1361687,2015-09-11 20:49:49.439,0,view,NaN,True,0_1
1367212,2015-09-11 20:52:39.591,0,view,170.152,False,0_1
1367342,2015-09-11 20:55:17.175,0,view,157.584,False,0_1
830385,2015-08-13 17:46:06.444,1,view,NaN,True,1_1
742616,2015-08-07 17:51:44.567,2,view,NaN,True,2_1
735273,2015-08-07 17:53:33.790,2,view,109.223,False,2_1
737711,2015-08-07 17:56:52.664,2,view,198.874,False,2_1
726292,2015-08-07 18:01:08.920,2,view,256.256,False,2_1
737615,2015-08-07 18:08:25.669,2,view,436.749,False,2_1
735202,2015-08-07 18:17:24.375,2,view,538.706,False,2_1


### Feature Engineering for Clustering



In [12]:
print("Starting lookup map creation...")

# 1. Item-to-Category Map
# We need a simple map of [itemid -> categoryid] to enrich both our
# session features and our market baskets.
print("Creating Item-to-Category map...")
item_category_map = df_properties[df_properties['property'] == 'categoryid'].copy()

# The properties file can have multiple categories for the same item over time.
# For simplicity, we'll take the *most recent* category entry for each item.
item_category_map.sort_values(by=['itemid', 'timestamp'], ascending=[True, False], inplace=True)
item_category_map = item_category_map.drop_duplicates(subset='itemid', keep='first')

# Convert value to int for clean merging
item_category_map['categoryid'] = item_category_map['value'].astype(int)
item_category_map = item_category_map[['itemid', 'categoryid']]

print(f"Created item_category_map with {len(item_category_map):,} unique item-category pairs.")

# 2. Item Availability Map
# This map will tell us if an item was in stock at the time
# of an event. This is tricky because availability changes over time.
print("Creating Item Availability map...")
item_availability_map = df_properties[df_properties['property'] == 'available'].copy()

item_availability_map['is_available'] = item_availability_map['value'].astype(int)
item_availability_map['timestamp_dt'] = pd.to_datetime(item_availability_map['timestamp'], unit='ms')

item_availability_map = item_availability_map[['itemid', 'timestamp_dt', 'is_available']]

# Sort by the keys we will merge on
item_availability_map.sort_values(by=['itemid', 'timestamp_dt'], inplace=True)
print(f"Created item_availability_map with {len(item_availability_map):,} availability records.")

category_name_map = df_categories[['categoryid']].copy()
# We could try to join parent names later if needed.
print(f"Created category_name_map with {len(category_name_map):,} categories.")

print("\nLookup map creation complete.")
print("The key dataframes for the next step are: 'item_category_map' and 'item_availability_map'.")

Starting lookup map creation...
Creating Item-to-Category map...
Created item_category_map with 417,053 unique item-category pairs.
Creating Item Availability map...
Created item_availability_map with 1,503,639 availability records.
Created category_name_map with 1,669 categories.

Lookup map creation complete.
The key dataframes for the next step are: 'item_category_map' and 'item_availability_map'.


In [13]:
print("Starting rich feature engineering...")
print(f"Original df_events shape: {df_events.shape}")

# 1. Enrich Events
# We add time-based features (day/hour) from the timestamp.
# These will be aggregated to find time-of-day or day-of-week patterns.
print("Extracting time-based features (hour, day)...")
df_events['session_hour_of_day'] = df_events['timestamp_dt'].dt.hour
df_events['session_day_of_week'] = df_events['timestamp_dt'].dt.dayofweek

# We do a left merge to add 'categoryid' to every event.
# Events for items not in the properties file will have NaN.
print("Enriching events with 'categoryid'...")
df_events_rich = pd.merge(df_events, item_category_map, on='itemid', how='left')

# This is the time-sensitive merge. We use pd.merge_asof()
# to join each event with the *last known availability* status *before*
# that event occurred.
# We must sort both dataframes by the merge keys ('itemid', 'timestamp_dt').
print("Enriching events with 'is_available' (using merge_asof)...")
df_events_rich = pd.merge_asof(
    df_events_rich.sort_values('timestamp_dt'),
    item_availability_map.sort_values('timestamp_dt'),
    on='timestamp_dt',
    by='itemid',
    direction='backward' # Finds the last known value before the event
)

# After the merge, 'is_available' will be NaN for items with no
# availability data. We'll assume these items are unavailable (0).
df_events_rich['is_available'] = df_events_rich['is_available'].fillna(0).astype(int)


# 2. Aggregate Session-Level Features
# Now we group by 'session_id' and aggregate all event-level
# data into a single row per session.
print("Aggregating features by session_id...")
agg_dict = {
    'timestamp_dt': [lambda x: (x.max() - x.min()).total_seconds(), 'min'], # Duration and start time
    'event': 'count',                 # total_events
    'itemid': 'nunique',              # unique_items_viewed
    'categoryid': 'nunique',          # unique_categories_viewed
    'is_available': 'mean',           # avg_item_availability
    'session_hour_of_day': 'first',   # Time of session start
    'session_day_of_week': 'first'    # Day of session start
}

session_features = df_events_rich.groupby('session_id').agg(agg_dict)

# Flatten the multi-index columns created by .agg()
session_features.columns = ['session_duration_sec', 'session_start_time',
                            'total_events', 'unique_items_viewed',
                            'unique_categories_viewed', 'avg_item_availability',
                            'session_hour_of_day', 'session_day_of_week']


# 3. Pivot Event Counts
#  We need specific counts for 'view', 'addtocart', and 'transaction'
# to build our ratio features and identify buyers.
print("Pivoting event counts...")
event_counts = df_events_rich.groupby('session_id')['event'].value_counts().unstack(fill_value=0)

# Rename columns for clarity.
event_counts.columns = [col + '_count' for col in event_counts.columns]

# Ensure all three columns exist, even if one event type (like 'transaction')
# didn't happen in any session (unlikely, but safe)
for col in ['view_count', 'addtocart_count', 'transaction_count']:
    if col not in event_counts.columns:
        event_counts[col] = 0

# Create our key target variable, 'is_buyer'.
event_counts['is_buyer'] = (event_counts['transaction_count'] > 0).astype(int)


# 4. Combine and Create Final Ratio Features
#  Merge our two aggregated dataframes to create the final matrix.
print("Combining aggregated dataframes...")
session_feature_matrix = pd.merge(session_features, event_counts, on='session_id', how='left')

# Create the final ratio features based on our plan.
# We handle division by zero (e.g., sessions with 0 duration or 0 views)
# by replacing resulting 'inf' or 'NaN' values with 0.
print("Creating final ratio features...")

# Event rate (how fast the user is clicking)
session_feature_matrix['event_rate_per_sec'] = (
    session_feature_matrix['total_events'] / (session_feature_matrix['session_duration_sec'] + 1) # +1 to avoid 0s
).fillna(0)

# View-to-cart ratio (micro-conversion)
session_feature_matrix['view_to_cart_ratio'] = (
    session_feature_matrix['addtocart_count'] / session_feature_matrix['view_count']
).fillna(0).replace(np.inf, 0)

# Category spread (focused vs. broad browser)
session_feature_matrix['category_spread_ratio'] = (
    session_feature_matrix['unique_categories_viewed'] / session_feature_matrix['unique_items_viewed']
).fillna(0).replace(np.inf, 0)

# Is weekend
session_feature_matrix['is_weekend'] = (
    session_feature_matrix['session_day_of_week'].isin([5, 6])
).astype(int)

# 5. Report Results
print("\n Feature Engineering Summary ")
print(f"Final session_feature_matrix shape: {session_feature_matrix.shape}")
print("\nFinal Matrix Info:")
session_feature_matrix.info()
print("\nSample of final feature matrix (first 5 rows):")
print(session_feature_matrix.head())


Starting rich feature engineering...
Original df_events shape: (2756101, 7)
Extracting time-based features (hour, day)...
Enriching events with 'categoryid'...
Enriching events with 'is_available' (using merge_asof)...
Aggregating features by session_id...
Pivoting event counts...
Combining aggregated dataframes...
Creating final ratio features...

 Feature Engineering Summary 
Final session_feature_matrix shape: (1761675, 16)

Final Matrix Info:
<class 'pandas.core.frame.DataFrame'>
Index: 1761675 entries, 0_1 to 9_1
Data columns (total 16 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   session_duration_sec      float64       
 1   session_start_time        datetime64[ns]
 2   total_events              int64         
 3   unique_items_viewed       int64         
 4   unique_categories_viewed  int64         
 5   avg_item_availability     float64       
 6   session_hour_of_day       int32         
 7   session_day_of_week     

In [ ]:
# Features
print("Starting rich feature engineering...")
print(f"Original df_events shape: {df_events.shape}")

# 1. Enrich Events
# We add time-based features (day/hour) from the timestamp.
# These will be aggregated to find time-of-day or day-of-week patterns.
print("Extracting time-based features (hour, day)...")
df_events['session_hour_of_day'] = df_events['timestamp_dt'].dt.hour
df_events['session_day_of_week'] = df_events['timestamp_dt'].dt.dayofweek

# We do a left merge to add 'categoryid' to every event.
# Events for items not in the properties file will have NaN.
print("Enriching events with 'categoryid'...")
df_events_rich = pd.merge(df_events, item_category_map, on='itemid', how='left')

# This is the time-sensitive merge. We use pd.merge_asof()
# to join each event with the *last known availability* status *before*
# that event occurred.
# We must sort both dataframes by the merge keys ('itemid', 'timestamp_dt').
print("Enriching events with 'is_available' (using merge_asof)...")
df_events_rich = pd.merge_asof(
    df_events_rich.sort_values('timestamp_dt'),
    item_availability_map.sort_values('timestamp_dt'),
    on='timestamp_dt',
    by='itemid',
    direction='backward' # Finds the last known value before the event
)

# After the merge, 'is_available' will be NaN for items with no
# availability data. We'll assume these items are unavailable (0).
df_events_rich['is_available'] = df_events_rich['is_available'].fillna(0).astype(int)

# 2. Aggregate Session-Level Features
# Now we group by 'session_id' and aggregate all event-level
# data into a single row per session.
print("Aggregating features by session_id...")
agg_dict = {
    'timestamp_dt': [lambda x: (x.max() - x.min()).total_seconds(), 'min'], # Duration and start time
    'event': 'count',                 # total_events
    'itemid': 'nunique',              # unique_items_viewed
    'categoryid': 'nunique',          # unique_categories_viewed
    'is_available': 'mean',           # avg_item_availability
    'session_hour_of_day': 'first',   # Time of session start
    'session_day_of_week': 'first'    # Day of session start
}

session_features = df_events_rich.groupby('session_id').agg(agg_dict)

# Flatten the multi-index columns created by .agg()
session_features.columns = ['session_duration_sec', 'session_start_time',
                            'total_events', 'unique_items_viewed',
                            'unique_categories_viewed', 'avg_item_availability',
                            'session_hour_of_day', 'session_day_of_week']


# 3. Pivot Event Counts ---
#  We need specific counts for 'view', 'addtocart', and 'transaction'
# to build our ratio features and identify buyers.
print("Pivoting event counts...")
event_counts = df_events_rich.groupby('session_id')['event'].value_counts().unstack(fill_value=0)

#  Rename columns for clarity.
event_counts.columns = [col + '_count' for col in event_counts.columns]

# Ensure all three columns exist, even if one event type (like 'transaction')
# didn't happen in any session (unlikely, but safe)
for col in ['view_count', 'addtocart_count', 'transaction_count']:
    if col not in event_counts.columns:
        event_counts[col] = 0

#  Create our key target variable, 'is_buyer'.
event_counts['is_buyer'] = (event_counts['transaction_count'] > 0).astype(int)


# --- 4. Combine and Create Final Ratio Features ---
#  Merge our two aggregated dataframes to create the final matrix.
print("Combining aggregated dataframes...")
session_feature_matrix = pd.merge(session_features, event_counts, on='session_id', how='left')

#  Create the final ratio features based on our plan.
# We handle division by zero (e.g., sessions with 0 duration or 0 views)
# by replacing resulting 'inf' or 'NaN' values with 0.
print("Creating final ratio features...")

# Event rate (how fast the user is clicking)
session_feature_matrix['event_rate_per_sec'] = (
    session_feature_matrix['total_events'] / (session_feature_matrix['session_duration_sec'] + 1) # +1 to avoid 0s
).fillna(0)

# View-to-cart ratio (micro-conversion)
session_feature_matrix['view_to_cart_ratio'] = (
    session_feature_matrix['addtocart_count'] / session_feature_matrix['view_count']
).fillna(0).replace(np.inf, 0)

# Category spread (focused vs. broad browser)
session_feature_matrix['category_spread_ratio'] = (
    session_feature_matrix['unique_categories_viewed'] / session_feature_matrix['unique_items_viewed']
).fillna(0).replace(np.inf, 0)

# Is weekend
session_feature_matrix['is_weekend'] = (
    session_feature_matrix['session_day_of_week'].isin([5, 6])
).astype(int)

# --- 5. Report Results ---
print("\n--- Feature Engineering Summary ---")
print(f"Final session_feature_matrix shape: {session_feature_matrix.shape}")
print("\nFinal Matrix Info:")
session_feature_matrix.info()
print("\nSample of final feature matrix (first 5 rows):")
print(session_feature_matrix.head())

display(session_feature_matrix.head())

session_feature_matrix.to_csv(f'{OUTPUTS_DIR}/session_feature_matrix.csv')
print(f"Features session matrix data saved to '{OUTPUTS_DIR}/session_feature_matrix.csv'.")

,session_duration_sec,session_start_time,total_events,unique_items_viewed,unique_categories_viewed,avg_item_availability,session_hour_of_day,session_day_of_week,addtocart_count,transaction_count,view_count,is_buyer,event_rate_per_sec,view_to_cart_ratio,category_spread_ratio,is_weekend
session_id,,,,,,,,,,,,,,,,
0_1,327.736,2015-09-11 20:49:49.439,3,3,3,0.333333,20,4,0,0,3,0,0.009126,0.0,1.000000,0
1000000_1,0.000,2015-06-05 18:16:10.629,1,1,1,0.000000,18,4,0,0,1,0,1.000000,0.0,1.000000,0
1000001_1,0.000,2015-07-07 18:12:14.953,1,1,1,0.000000,18,1,0,0,1,0,1.000000,0.0,1.000000,0
1000001_2,1061.726,2015-07-24 20:18:15.303,3,3,2,1.000000,20,4,0,0,3,0,0.002823,0.0,0.666667,0
1000001_3,0.000,2015-07-29 20:38:29.170,1,1,1,1.000000,20,2,0,0,1,0,1.000000,0.0,1.000000,0


Features session matrix data saved to '/content/drive/MyDrive/UP Honours/COS781 -Data Mining/Project/session_feature_matrix.csv'.


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load Data
# Assuming 'session_feature_matrix' is in memory from Step 4.

print("Starting data scaling for K-Means...")

# 1. Select Features for Clustering
#  We will select only the numerical features that describe
# session *behavior*. We exclude non-behavioral columns like
# 'session_start_time' and 'is_buyer' (which we'll use later for profiling,
# not for the clustering itself).
features_for_clustering = [
    'session_duration_sec',
    'total_events',
    'unique_items_viewed',
    'unique_categories_viewed',
    'avg_item_availability',
    'session_hour_of_day',
    'view_count',
    'addtocart_count',
    'event_rate_per_sec',
    'view_to_cart_ratio',
    'category_spread_ratio',
    'is_weekend'
]

# Create the final matrix for clustering
X = session_feature_matrix[features_for_clustering]

#  Handle any potential inf/-inf values that might have resulted
# from division, just in case.
X.replace([np.inf, -np.inf], 0, inplace=True)


# 2. Scale the Data
#  We use StandardScaler to transform the data so that each
# feature has a mean of 0 and a standard deviation of 1. This
# standardizes their importance for the K-Means algorithm.
print("Scaling features using StandardScaler...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to a DataFrame for easier handling
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# 3. Report Results
print("\n--- Scaling Summary ---")
print(f"Data scaled successfully. Shape: {X_scaled_df.shape}")
print("\nSample of scaled data:")
print(X_scaled_df.head())


# Save the scaled data for the next notebook
X_scaled_df.to_csv(f'{OUTPUTS_DIR}/session_feature_matrix_scaled.csv')
print(f"Scaled data saved to '{OUTPUTS_DIR}/session_feature_matrix_scaled.csv'.")

Starting data scaling for K-Means...


/tmp/ipython-input-2304352175.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], 0, inplace=True)


Scaling features using StandardScaler...

--- Scaling Summary ---
Data scaled successfully. Shape: (1722864, 12)

Sample of scaled data:
            session_duration_sec  total_events  unique_items_viewed  \
session_id                                                            
0_1                     0.747928      1.016926             1.725313   
1000000_1              -0.269207     -0.314189            -0.269693   
1000001_1              -0.269207     -0.314189            -0.269693   
1000001_2               3.025881      1.016926             1.725313   
1000001_3              -0.269207     -0.314189            -0.269693   

            unique_categories_viewed  avg_item_availability  \
session_id                                                    
0_1                         4.372708              -0.127893   
1000000_1                   0.147813              -0.827871   
1000001_1                   0.147813              -0.827871   
1000001_2                   2.260261              

In [ ]:
# 1. Session Duration Distribution
sns.histplot(session_feature_matrix['session_duration_sec'], bins=50, kde=True)
plt.title("Distribution of Session Duration (sec)")
plt.xlabel("Session Duration (sec)"); plt.ylabel("Frequency")
plt.show()

# 2. Session Start Time Heatmap
pd.crosstab(session_feature_matrix['session_day_of_week'], session_feature_matrix['session_hour_of_day']) \
  .pipe(lambda df: sns.heatmap(df, cmap="YlGnBu"))
plt.title("Sessions by Hour and Day of Week")
plt.show()

# 3. Buyer vs. Non-Buyer Boxplot
sns.boxplot(x="is_buyer", y="view_to_cart_ratio", data=session_feature_matrix)
plt.title("View-to-Cart Ratio: Buyer vs. Non-Buyer")
plt.show()

# 4. Correlation Heatmap
corr = session_feature_matrix.corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Feature Correlation Heatmap")
plt.show()